# The MultiIndex object

## View vs copy

In [1]:
import pandas as pd

In [2]:
mlo = pd.read_csv('../data/co2-mm-mlo.csv', na_values=-99.99, index_col='Date', parse_dates=True)

In [3]:
mlo.head()

,Decimal Date,Average,Interpolated,Trend,Number of Days
Date,,,,,
1958-03-01,1958.208,315.71,315.71,314.62,-1
1958-04-01,1958.292,317.45,317.45,315.29,-1
1958-05-01,1958.375,317.50,317.50,314.71,-1
1958-06-01,1958.458,NaN,317.10,314.85,-1
1958-07-01,1958.542,315.86,315.86,314.98,-1


In [4]:
s = mlo['Interpolated']

In [5]:
mlo.assign(smooth=s.rolling(12).mean()).tail()

,Decimal Date,Average,Interpolated,Trend,Number of Days,smooth
Date,,,,,,
2016-08-01,2016.625,402.25,402.25,404.09,23,403.157500
2016-09-01,2016.708,401.03,401.03,404.52,24,403.440833
2016-10-01,2016.792,401.57,401.57,404.93,29,403.714167
2016-11-01,2016.875,403.53,403.53,405.57,27,403.995000
2016-12-01,2016.958,404.48,404.48,405.25,29,404.214167


A copy is returned.

In [6]:
mlo.head()

,Decimal Date,Average,Interpolated,Trend,Number of Days
Date,,,,,
1958-03-01,1958.208,315.71,315.71,314.62,-1
1958-04-01,1958.292,317.45,317.45,315.29,-1
1958-05-01,1958.375,317.50,317.50,314.71,-1
1958-06-01,1958.458,NaN,317.10,314.85,-1
1958-07-01,1958.542,315.86,315.86,314.98,-1


In [7]:
s2 = mlo.loc[:'1958-05', 'Average']
s2

Date
1958-03-01    315.71
1958-04-01    317.45
1958-05-01    317.50
Name: Average, dtype: float64

A view is returned.

In [8]:
s2[:] = 313

In [9]:
s2

Date
1958-03-01    313.0
1958-04-01    313.0
1958-05-01    313.0
Name: Average, dtype: float64

In [10]:
mlo.head()

,Decimal Date,Average,Interpolated,Trend,Number of Days
Date,,,,,
1958-03-01,1958.208,313.00,315.71,314.62,-1
1958-04-01,1958.292,313.00,317.45,315.29,-1
1958-05-01,1958.375,313.00,317.50,314.71,-1
1958-06-01,1958.458,NaN,317.10,314.85,-1
1958-07-01,1958.542,315.86,315.86,314.98,-1


### Hands-on exercise

1. How could you create a series equal to `s2` while preserving the original `mlo` DataFrame (Hint: Remember the NumPy lesson.)

## Chained indexing

In [11]:
mlo['Average']['1958-03']

Date
1958-03-01    313.0
Name: Average, dtype: float64

In [12]:
mlo['Average']['1958-03'] = 312

/home/marianne/python/miniconda3/envs/advanced-pandas/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Generally speaking, chained indexing is not a good practice. To set a new value, use `mlo.loc[row_indexer, col_indexer]` because `mlo.loc` is guaranteed to be `mlo` itself.

In [13]:
mlo.loc['1958-03', 'Average']

Date
1958-03-01    312.0
Name: Average, dtype: float64

## Hierarchical indexing

## Reshaping

## Pivoting